In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from nltk.chat.util import  reflections
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import requests
import re
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [4]:
questions = []
answers = []
url = 'https://xatonline.in/xat-faqs.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')

In [5]:
qnas = soup.find_all(id=re.compile(re.compile(r'accordion.')))

In [6]:
qnas

[<div class="panel-group accordian-mob" id="accordion1"><div class="panel panel-default"><div class="panel-heading" style="padding: 0"><h4 class="panel-title" style="display: flex;"><a class="collapsed" data-parent="#accordion1" data-toggle="collapse" href="#course1_1" style="padding: 10px 15px; display:flex; flex-grow: 1;">Q. 1 How do I know that my application is registered?</a></h4></div><div class="panel-collapse collapse" id="course1_1"><div class="panel-body"><div class="container-fluid"><div class="row"><p>Successful Registration is indicated by the Page displayed after clicking the Register Button indicating that a verification Email has been triggered to your registered Email ID, you need to click on the verification Email and you'll receive a credential mail post that.</p></div></div></div></div></div><div class="panel panel-default"><div class="panel-heading" style="padding: 0"><h4 class="panel-title" style="display: flex;"><a class="collapsed" data-parent="#accordion1" data

In [7]:
for qna in qnas:
    
    que = qna.find_all('h4')
    for q in que:
        w = re.split(r'Q. . ',q.text)[1]
        questions.append(w)
    ans = qna.find_all('p')
    for a in ans:
        a = a.text
        answers.append(a)

In [8]:
len(questions), len(answers)

(30, 30)

In [9]:
questions = np.array(questions).flatten()

In [10]:
pd.DataFrame({'Questions':questions,'Answers':answers})

,Questions,Answers
0,How do I know that my application is registered?,Successful Registration is indicated by the Pa...
1,I did not receive the email intimation for reg...,Delivery of these Emails and SMS purely depend...
2,"I have forgotten my XAT password, could you help!","Navigate to website, click on ""Login"" and then..."
3,When shall I receive the XAT 2021 Admit Card?,You can download the admit card from our websi...
4,What is the duration of the test?,The duration of the test will be of three hour...
5,When are XAT 2021 results out?,The XAT Result will be declared 3 weeks after ...
6,Will there be any sectional time limits or ove...,"There will be only overall time limits, there ..."
7,Will the candidates have access to an on scree...,"No, the candidates will not be given access to..."
8,Will the candidates be able to choose the orde...,Yes the candidates can choose any section to b...
9,Can the candidates switch between sections - h...,Yes.


In [11]:
def get_unique(array):
    unique_list = []
    for x in array:
        if x not in unique_list:
            unique_list.append(x)
            
    return unique_list

In [12]:
lemmatizer = WordNetLemmatizer()

def preprocess(sent):
    
    not_words = ['who','where','what','why','where','how','i','can','when','must','should','get']
    
    #Removing Punctuations
    for p in list(punctuation):
        sent = sent.replace(p,' ')
        
    sent = sent.lower().split()
    
    #Removing stopwords and unnecessary words
    stop_words = set(stopwords.words('english'))
    sent = [i for i in sent if i not in stop_words]
    sent = [i for i in sent if i not in not_words]
    
    #Lemmatizing the words
    sent = [lemmatizer.lemmatize(item, pos='v') for item in sent]
    
    return get_unique(sent)

In [13]:
# This bot quits when q is given as the input
def ask_me(response):
     
    while(1):
        
        if response.lower() == 'q':
            print("Quitting the bot...")
            break  
        
        i = 0
        chosen = len(questions)
        matches = 0
        list_response = preprocess(response)

        while i < len(questions):
            loc_matches = 0
            x = questions[i] + ' '.join(answers[i])
            list_x = preprocess(x)

            for word in list_x:
                if word in list_response:
                    loc_matches += 1

            if loc_matches > matches:
                chosen = i
                matches = loc_matches
            i = i + 1
    
        if chosen < len(questions):
            ans = answers[chosen]
            print(ans)
        else:
            print("I am unable to answer to your question. Please reach out to support@website.com")
        break

In [14]:
ask_me("I forgot my password")

Navigate to website, click on "Login" and then "Forgot Password", follow the steps to change the password.


In [15]:
ask_me("What is the duration of the test")

The duration of the test will be of three hours i.e. from 9:30 AM to 12.30 PM on January 03, 2021, Sunday.


In [16]:
ask_me("When will the results be out?")

The XAT Result will be declared 3 weeks after the conduct of XAT-2021


In [17]:
ask_me("Can I answer sections in any order?")

Yes the candidates can choose any section to begin with.


In [18]:
ask_me("How to change some of the details in my application")

We do not entertain any changes in the Application form until it's very crucial. For that matter you need to raise a query by clicking "Any Query? Ask us" on your dashboard.


In [19]:
ask_me("How to calculate arks for CBSE?")

The candidate should take the aggregate of all the subjects for 10th.


In [20]:
ask_me("I am unable to make payment")

Online payments involve multiple agencies and there are chances that your payment is timed out or declined. If you encounter any such problems, we would suggest you to try again with a different debit/credit card or net banking option available and a different browser. If you still face the issue drop a query by clicking on "Any Query, Ask us" on your dashboard and share with us the screenshot of the error page, for better understanding of the issue. We will get back to you with a resolution at the earliest.


In [21]:
ask_me("What is the eligibility?")

Recognized Bachelor’s Degree of minimum three years duration or equivalent in any discipline. Those completing their final examination by June 10, 2021 may also apply. All Indian candidates must appear for XAT 2021. However, NRI and Foreign candidates may apply through GMAT score.


In [22]:
ask_me("Can I include internships in my work experience?")

Internship/Articleship is not counted as work experience.


In [23]:
ask_me(" Is this easier than CAT?")

I am unable to answer to your question. Please reach out to support@website.com


In [24]:
ask_me("Is Xavier Institute of Management better than IIM A?")

Candidates interested in XLRI need to apply for the programmes of their choice. Please note that each institution independently decides the weightage to be given for each of the test components on the basis of its need. XLRI does not play any role in selection of the candidates in these institutes for Xat Associate Member Institutes click here . XAT is a common test for admission into academic programmes of XLRI.


In [25]:
ask_me("What is the NIRF ranking for XLRI?")

Candidates interested in XLRI need to apply for the programmes of their choice. Please note that each institution independently decides the weightage to be given for each of the test components on the basis of its need. XLRI does not play any role in selection of the candidates in these institutes for Xat Associate Member Institutes click here . XAT is a common test for admission into academic programmes of XLRI.


In [26]:
ask_me("Can I use calculator?")

No, the candidates will not be given access to on screen calculator


In [27]:
ask_me("Q")

Quitting the bot...


In [28]:
ask_me("can I get the contact number of the Dean?")

I am unable to answer to your question. Please reach out to support@website.com


In [29]:
ask_me("What is the hostel fee for the 2 year programme?")

Yes, you are eligible for admission to XLRI. You need to complete your final year examination by June 10, 2021.


As we can see, this chatbot is unable to understand the context but can match the words and can match the words with the already available questions to provide the answer. 

I will try to design a chatbot using CountVectorizer.

In [30]:
def preprocess_tfidf(sent):
    
    not_words = ['who','where','what','why','where','how','i','can','when','must','should','get']
    
    #Removing Punctuations
    for p in list(punctuation):
        sent = sent.replace(p,' ')
        
    sent = sent.lower().split()
    
    #Removing stopwords and unnecessary words
    stop_words = set(stopwords.words('english'))
    sent = [i for i in sent if i not in stop_words]
    sent = [i for i in sent if i not in not_words]
    
    #Lemmatizing the words
    sent = [lemmatizer.lemmatize(item, pos='v') for item in sent]
    
    return ' '.join(get_unique(sent))

In [31]:
def tfidf_bot(string):
    count = CountVectorizer()
    
    count_questions = count.fit_transform([preprocess_tfidf(question) for question in questions])
    count_string = count.transform([preprocess_tfidf(string)])
    
    
    cosine_sim = cosine_similarity(count_string,count_questions)
    
    max_cosine = np.max(cosine_sim)
    index = np.argmax(cosine_sim)

    if max_cosine >= 0.35:
        print(answers[index])
    else:
        print("I am unable to answer to your question. Please reach out to support@website.com")

In [32]:
tfidf_bot("Can I use calculator?")

No, the candidates will not be given access to on screen calculator


In [33]:
tfidf_bot("What is the NIRF ranking for XLRI?")

To Know More click here 


In [34]:
tfidf_bot("I forgot my password")

Navigate to website, click on "Login" and then "Forgot Password", follow the steps to change the password.


In [35]:
tfidf_bot("Is this easier than CAT?")

I am unable to answer to your question. Please reach out to support@website.com


In [36]:
tfidf_bot("How should I calculate percentage for CBSE?")

The candidate should take the aggregate of all the subjects for 10th.


In [37]:
tfidf_bot("Can I include internships in my work experience?")

No, only the work experience post the graduation is counted.


In [38]:
tfidf_bot("Can I answer sections in any order?")

Yes the candidates can choose any section to begin with.


In [39]:
tfidf_bot("How to change some of the details in my application")

We do not entertain any changes in the Application form until it's very crucial. For that matter you need to raise a query by clicking "Any Query? Ask us" on your dashboard.


In [40]:
tfidf_bot("Is XLRI better than IIM A?")

To Know More click here 


In [41]:
tfidf_bot("How much is the fees?")

I am unable to answer to your question. Please reach out to support@website.com


In [42]:
tfidf_bot("can I get the contact number of the Dean?")

I am unable to answer to your question. Please reach out to support@website.com


In [43]:
tfidf_bot("When will the results be out?")

The XAT Result will be declared 3 weeks after the conduct of XAT-2021


In [44]:
tfidf_bot("Is XAT easier than CAT?")

The XAT Result will be declared 3 weeks after the conduct of XAT-2021


In [45]:
tfidf_bot("How much is the fees for XLRI HR programme?")

A candidate can apply to XLRI Delhi BM Programme by clicking on BM programme and giving the preference as Delhi. There is no extra fee charged .


In [46]:
tfidf_bot("What is the average placement at XLRI?")

To Know More click here 


In [47]:
tfidf_bot("What are the sections in the exam?")

All three sections will be open to the candidate once the exam starts.


In [48]:
tfidf_bot("Where is XLRI?")

To Know More click here 


In [49]:
tfidf_bot("Can I use this core to get admission in any other institutes?")

I am unable to answer to your question. Please reach out to support@website.com


Eventhough the CountVectorizer method is better than the manual one above, the results are not satisfactory.